In [1]:
import numpy as np
import scipy.integrate
from matplotlib import pyplot as plt
from tabulate import tabulate 
import glob as glob

In [6]:
def getew(glbdata,tex = False):
    ''' This function takes a list file (of text spectra) and outputs an array of equivolent widths, and their associated errors.
    Optionally it can also output this array in TeX format. This can be called by setting tex=True, and providing an additional
    object when calling the function.'''
    spectra = []
    for datafile in glbdata:
        spectra.append(np.loadtxt(datafile))
  
    size = len(glbdata)
    ewdata = np.zeros([size,8]) #8 w/out CO, 10 with it
    filenumber = 0
    # The loop below finds given features in the input spectra, and calculates the EW
    for file in spectra:

        startH2a = 297
        endH2a = 303 
        count1 = 0
        count2 = 0
        count3 = 0
        # Below finds the exact width of any feature in the expected range 
        while startH2a > 293 and startH2a < 299 and count1 <10:
            if file[startH2a][1] > file[startH2a + 1][1]:
                startH2a+= 1 

            elif file[startH2a][1] > file[startH2a -1][1]:
                startH2a-= 1

            count1+= 1


        while endH2a > 300 and endH2a < 306 and count2 <10:
            if file[endH2a][1] > file[endH2a - 1][1]:
                endH2a-= 1

            elif file[endH2a][1] > file[endH2a + 1][1]:
                endH2a+= 1

            count2+= 1
        # This loop is a contingency incase the shape of the the continuum 
        # causes the feauture size to be miscalculated     
        while count3 < 3:
            testgrad1 = (file[endH2a + 1][1] - file[endH2a][1]) / (file[endH2a + 1][0] - file[endH2a][0])   
            testgrad2 = (file[endH2a][1] - file[endH2a - 1][1]) / (file[endH2a][0] - file[endH2a - 1][0])
            if np.absolute(testgrad1) > np.absolute(testgrad2):
                testgradhigh = testgrad1
                testgradlow = testgrad2
        
            elif np.absolute(testgrad2) > np.absolute(testgrad1):
                testgradhigh = testgrad2
                testgradlow = testgrad1
            
            if (1.6 * np.absolute(testgradhigh)) > np.absolute(testgradlow) and np.absolute(testgradlow) > (0.4 * np.absolute(testgradhigh)):
                endH2a -= 1
        
            count3 += 1
            
        # Now read only the data pertaining to the emission/absorbtion

        H2a = file[startH2a:endH2a + 1]
        xdatH2a = H2a[:,0]
        ydatH2a = H2a[:,1]
        last = (len(ydatH2a))-1
        grad = (ydatH2a[last] - ydatH2a[0]) / (xdatH2a[last] - np.amin(xdatH2a))
        c = ydatH2a[0] - (grad*xdatH2a[0])
        norm = [((grad*i) + c) for i in xdatH2a]      
        ydatH2an = ydatH2a - norm 
        
        #Analyse feature
        
        area = scipy.integrate.cumtrapz(ydatH2an,xdatH2a)
        height = np.amax(ydatH2a)

        #find EW 

        if np.min(norm) < np.median(ydatH2a):
            ew = -1 * (area[last - 1] / height)
        else:
            ew = area[last -1] / height

        #do the errors 

        l = len(ydatH2a)
        sigarr = np.zeros((2,l))
        sigarr[0,:] = xdatH2a[:]
        sigarr[1,:] = ydatH2a[:]
        sigy = np.std(ydatH2a)/np.sqrt(l)
        sigx = np.std(xdatH2a)/np.sqrt(l)
        sigA = (area[last - 1]/height)*np.sqrt((sigx/(xdatH2a[last]-xdatH2a[0]))**2)

        if np.abs(ew) < 1.2:

            ew = 0
            sigA = 0

        ewdata[filenumber,0] = ew 
        ewdata[filenumber,1] = sigA
        
        # Brackett Gamma
        
        startBG = 366
        endBG = 376 
        count1 = 0
        count2 = 0
        count3 = 0
        while startBG > 360 and startBG < 371 and count1 <10:
            if file[startBG][1] > file[startBG + 1][1]:
                startBG+= 1 

            elif file[startBG][1] > file[startBG -1][1]:
                startBG-= 1

            count1+= 1


        while endBG > 373 and endBG < 385 and count2 <10:
            if file[endBG][1] > file[endBG - 1][1]:
                endBG-= 1

            elif file[endBG][1] > file[endBG + 1][1]:
                endBG+= 1

            count2+= 1
        
        while count3 < 3:
            testgrad1 = (file[endBG + 1][1] - file[endBG][1]) / (file[endBG + 1][0] - file[endBG][0])   
            testgrad2 = (file[endBG][1] - file[endBG - 1][1]) / (file[endBG][0] - file[endBG - 1][0])
            if np.absolute(testgrad1) > np.absolute(testgrad2):
                testgradhigh = testgrad1
                testgradlow = testgrad2
        
            elif np.absolute(testgrad2) > np.absolute(testgrad1):
                testgradhigh = testgrad2
                testgradlow = testgrad1
            
            if (1.6 * np.absolute(testgradhigh)) > np.absolute(testgradlow) and np.absolute(testgradlow) > (0.4 * np.absolute(testgradhigh)):
                endBG -= 1
        
            count3 += 1

        BG = file[startBG:endBG + 1]
        xdatBG = BG[:,0]
        ydatBG = BG[:,1]
        last = (len(ydatBG))-1
        grad = (ydatBG[last] - ydatBG[0]) / (xdatBG[last] - np.amin(xdatBG)) 
        c = ydatBG[0] - (grad*xdatBG[0])
        norm = [((grad*i) + c) for i in xdatBG]      
        ydatBGn = ydatBG - norm 
        area = scipy.integrate.cumtrapz(ydatBGn,xdatBG)
        height = np.amax(ydatBG)

        if np.min(norm) < np.median(ydatBG):
            ew = -1 * (area[last - 1] / height)
        else:
            ew = area[last -1] / height

        l = len(ydatBG)
        sigarr = np.zeros((2,l))
        sigarr[0,:] = xdatBG[:]
        sigarr[1,:] = ydatBG[:]
        sigy = np.std(ydatBG)/np.sqrt(l)
        sigx = np.std(xdatBG)/np.sqrt(l)
        sigA = (area[last - 1]/height)*np.sqrt((sigx/(xdatBG[last]-xdatBG[0]))**2)

        ewdata[filenumber,2] = ew 
        ewdata[filenumber,3] = sigA
        
        # H2 (2.22)
        
        startH2b = 463
        endH2b = 469 
        count1 = 0
        count2 = 0
        count3 = 0
        while startH2b > 460 and startH2b < 466 and count1 <10:
            if file[startH2b][1] > file[startH2b + 1][1]:
                startH2b+= 1 

            elif file[startH2b][1] > file[startH2b -1][1]:
                startH2b-= 1

            count1+= 1


        while endH2b > 467 and endH2b < 471 and count2 <10:
            if file[endH2b][1] > file[endH2b - 1][1]:
                endH2b-= 1

            elif file[endH2b][1] > file[endH2b + 1][1]:
                endH2b+= 1

            count2+= 1
            
        while count3 < 3:
            testgrad1 = (file[endH2b + 1][1] - file[endH2b][1]) / (file[endH2b + 1][0] - file[endH2b][0])   
            testgrad2 = (file[endH2b][1] - file[endH2b - 1][1]) / (file[endH2b][0] - file[endH2b - 1][0])
            if np.absolute(testgrad1) > np.absolute(testgrad2):
                testgradhigh = testgrad1
                testgradlow = testgrad2
        
            elif np.absolute(testgrad2) > np.absolute(testgrad1):
                testgradhigh = testgrad2
                testgradlow = testgrad1
            
            if (1.6 * np.absolute(testgradhigh)) > np.absolute(testgradlow) and np.absolute(testgradlow) > (0.4 * np.absolute(testgradhigh)):
                endH2b -= 1
        
            count3 += 1

        H2b = file[startH2b:endH2b + 1]
        xdatH2b = H2b[:,0]
        ydatH2b = H2b[:,1]
        last = (len(ydatH2b))-1
        grad = (ydatH2b[last] - ydatH2b[0]) / (xdatH2b[last] - np.amin(xdatH2b))
        c = ydatH2b[0] - (grad*xdatH2b[0])
        norm = [((grad*i) + c) for i in xdatH2b]      
        ydatH2bn = ydatH2b - norm 
        area = scipy.integrate.cumtrapz(ydatH2bn,xdatH2b)

        #find height 

        height = np.amax(ydatH2b)

        #find EW 

        if np.min(norm) < np.median(ydatH2b):
            ew = -1 * (area[last - 1] / height)
        else:
            ew = area[last -1] / height

        #do the errors 

        l = len(ydatH2b)
        sigarr = np.zeros((2,l))
        sigarr[0,:] = xdatH2b[:]
        sigarr[1,:] = ydatH2b[:]
        sigy = np.std(ydatH2b)/np.sqrt(l)
        sigx = np.std(xdatH2b)/np.sqrt(l)
        sigA = (area[last - 1]/height)*np.sqrt((sigx/(xdatH2b[last]-xdatH2b[0]))**2)

        if np.abs(ew) < 1.2: 
            ew = 0
            sigA = 0                     

        ewdata[filenumber,4] = ew 
        ewdata[filenumber,5] = sigA

        # Sodium
        
        startNA = 433
        endNA = 437 
        count1 = 0
        count2 = 0
        count3 = 0
        while startNA > 432 and startNA < 436 and count1 <10:
            if file[startNA][1] > file[startNA + 1][1]:
                startNA+= 1 

            elif file[startNA][1] > file[startNA -1][1]:
                startNA-= 1

            count1+= 1


        while endNA > 436 and endNA < 440 and count2 <10:
            if file[endNA][1] > file[endNA - 1][1]:
                endNA-= 1

            elif file[endNA][1] > file[endNA + 1][1]:
                endNA+= 1

            count2+= 1
        
        while count3 < 3:
            testgrad1 = (file[endNA + 1][1] - file[endNA][1]) / (file[endNA + 1][0] - file[endNA][0])   
            testgrad2 = (file[endNA][1] - file[endNA - 1][1]) / (file[endNA][0] - file[endNA - 1][0])
            if np.absolute(testgrad1) > np.absolute(testgrad2):
                testgradhigh = testgrad1
                testgradlow = testgrad2
        
            elif np.absolute(testgrad2) > np.absolute(testgrad1):
                testgradhigh = testgrad2
                testgradlow = testgrad1
            
            if (1.6 * np.absolute(testgradhigh)) > np.absolute(testgradlow) and np.absolute(testgradlow) > (0.4 * np.absolute(testgradhigh)):
                endNA -= 1
        
            count3 += 1
        
        # Below is checking if the feaure is instead found in absortion
        
        if (endNA - startNA) < 3:
            startNA = 433
            endNA = 438
            count1 = 0
            count2 = 0
            while startNA > 432 and startNA < 436 and count1 <10:
                if file[startNA][1] < file[startNA + 1][1]:
                    startNA+= 1 

                elif file[startNA][1] < file[startNA -1][1]:
                    startNA-= 1

                count1+= 1


            while endNA > 436 and endNA < 440 and count2 <10:
                if file[endNA][1] < file[endNA - 1][1]:
                    endNA-= 1

                elif file[endNA][1] < file[endNA + 1][1]:
                    endNA+= 1

                count2+= 1
                
            while count3 < 3:
                testgrad1 = (file[endNA + 1][1] - file[endNA][1]) / (file[endNA + 1][0] - file[endNA][0])   
                testgrad2 = (file[endNA][1] - file[endNA - 1][1]) / (file[endNA][0] - file[endNA - 1][0])
                
                if np.absolute(testgrad1) > np.absolute(testgrad2):
                    testgradhigh = testgrad1
                    testgradlow = testgrad2

                elif np.absolute(testgrad2) > np.absolute(testgrad1):
                    testgradhigh = testgrad2
                    testgradlow = testgrad1

                if (1.6 * np.absolute(testgradhigh)) > np.absolute(testgradlow) and np.absolute(testgradlow) > (0.4 * np.absolute(testgradhigh)):
                    endNA -= 1

                count3 += 1

        NA = file[startNA:endNA + 1]
        xdatNA = NA[:,0]
        ydatNA = NA[:,1]
        last = (len(ydatNA))-1
        grad = (ydatNA[last] - ydatNA[0]) / (xdatNA[last] - np.amin(xdatNA))
        c = ydatNA[0] - (grad*xdatNA[0])
        norm = [((grad*i) + c) for i in xdatNA]      
        ydatNAn = ydatNA - norm 
        area = scipy.integrate.cumtrapz(ydatNAn,xdatNA)

        #find height 

        height = np.amax(ydatNA)

        #find EW 

        if np.min(norm) < np.median(ydatNA):
            ew = -1 * (area[last - 1] / height)
        else:
            ew = -1 * (area[last - 1] / height)

        #do the errors 

        l = len(ydatNA)
        sigarr = np.zeros((2,l))
        sigarr[0,:] = xdatNA[:]
        sigarr[1,:] = ydatNA[:]
        sigy = np.std(ydatNA)/np.sqrt(l)
        sigx = np.std(xdatNA)/np.sqrt(l)
        sigA = (area[last - 1]/height)*np.sqrt((sigx/(xdatNA[last]-xdatNA[0]))**2)

        # check if the feature is real or noise, noise should be set to 0.

        if np.abs(ew) < 1.2: 
            ew = 0
            sigA = 0                     

        ewdata[filenumber,6] = ew 
        ewdata[filenumber,7] = sigA

        filenumber+=1
        
    if tex == True: 
        table = tabulate(ewdata, tablefmt="latex", floatfmt=".4f")

    return ewdata,table        

In [10]:
alldata = glob.glob('spectra' + '/*_pub*')
x , y = getew(alldata, tex = True )
print(y)

\begin{tabular}{rrrrrrrr}
\hline
  0.0000 & 0.0000 &  -3.2655 & 0.4114 & -1.7920 & 0.2074 &  1.7065 & -0.2698 \\
  0.0000 & 0.0000 &  -6.5074 & 0.6205 &  0.0000 & 0.0000 &  0.0000 &  0.0000 \\
 -1.9632 & 0.3104 & -15.5271 & 1.4804 &  0.0000 & 0.0000 & -2.3667 &  0.2982 \\
  0.0000 & 0.0000 &  -4.2525 & 0.4575 & -1.7298 & 0.2412 &  0.0000 &  0.0000 \\
 -3.4636 & 0.3726 &  -6.9563 & 0.7484 &  0.0000 & 0.0000 &  0.0000 &  0.0000 \\
  0.0000 & 0.0000 &  -3.6329 & 0.3666 &  0.0000 & 0.0000 &  0.0000 &  0.0000 \\
  0.0000 & 0.0000 &  -7.7976 & 0.6213 &  0.0000 & 0.0000 &  0.0000 &  0.0000 \\
 -3.6153 & 0.3447 & -10.6475 & 0.8484 &  0.0000 & 0.0000 &  2.4155 & -0.2795 \\
  0.0000 & 0.0000 &  -6.1816 & 0.5600 &  0.0000 & 0.0000 &  0.0000 &  0.0000 \\
 -5.7096 & 0.6143 &  -7.0271 & 0.6700 &  0.0000 & 0.0000 &  0.0000 &  0.0000 \\
  0.0000 & 0.0000 &  -7.9746 & 0.7604 & -1.4238 & 0.1532 & -1.5216 &  0.1761 \\
 -1.9724 & 0.2485 &  -3.8172 & 0.4809 & -1.3062 & 0.1646 &  0.0000 &  0.0000 \\
  0.000